In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, Qwen2AudioForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info  

model_vl = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="cuda"
)
processor_vl = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

processor_audio = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model_audio = Qwen2AudioForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-Audio-7B-Instruct", device_map="cuda"
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

In [ ]:
video_path = "C:\\Users\\prath\\Desktop\\Confidence\\video_sample.mp4"


In [4]:

import cv2
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration = frame_count / fps if fps>0 else None
print(f"Video FPS: {fps}, Total frames: {frame_count}, Duration: {duration}s")
cap.release()


Video FPS: 30.0, Total frames: 609, Duration: 20.3s


In [5]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # GPU name


True
NVIDIA GeForce GTX 1650


In [8]:
messages_vl = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "C:\\Users\\prath\Desktop\\Confidence\\video_sample.mp4",
                "fps": 1.0  
            },
            {"type": "text", "text": " Based on the facial expression of this video, evaluate the confidence level of the person and return the output as only one label out of the following three: Low Confidence, Moderately Confident, Highly Confident."}
        ]
    }
]

text_prompt = processor_vl.apply_chat_template(messages_vl, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages_vl)
inputs_vl = processor_vl(text=[text_prompt], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt")
inputs_vl = inputs_vl.to("cuda")

gen_ids_vl = model_vl.generate(**inputs_vl, max_new_tokens=150)
gen_ids_vl = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs_vl.input_ids, gen_ids_vl)]
output_text_vl = processor_vl.batch_decode(gen_ids_vl, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print("Vision analysis:", output_text_vl)


c:\Users\prath\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:2504: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


In [ ]:

from moviepy.editor import VideoFileClip
clip = VideoFileClip(video_path)
audio_path = "temp_audio.wav"
clip.audio.write_audiofile(audio_path, logger=None)  

conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": f"file://{audio_path}"},
        {"type": "text", "text": "Based on this audio evaluate the confidence level of the person and return the output as only one label out of the following three: Low Confidence, Moderately Confident, Highly Confident. "}
    ]}
]

text_audio = processor_audio.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

import librosa
audio_wave, _ = librosa.load(audio_path, sr=processor_audio.feature_extractor.sampling_rate)
inputs_audio = processor_audio(text=[text_audio], audios=[audio_wave], return_tensors="pt", padding=True)
inputs_audio = inputs_audio.to("cuda")

gen_ids_audio = model_audio.generate(**inputs_audio, max_length=150)

gen_ids_audio = gen_ids_audio[:, inputs_audio.input_ids.size(1):]
output_text_audio = processor_audio.batch_decode(gen_ids_audio, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print("Audio analysis:", output_text_audio)


In [ ]:


print("Final confidence assessment (vision + audio):")
print("- From video (facial cues):", output_text_vl)
print("- From audio (vocal cues):", output_text_audio)


In [1]:
from transformers.utils import cached_file, hf_cache_home
import shutil
import os

# List of model names to delete
models = [
    "Qwen/Qwen2.5-VL-7B-Instruct",
    "Qwen/Qwen2-Audio-7B-Instruct"
]

# Hugging Face cache directory
cache_dir = os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "transformers")

# Delete model folders manually
for model_name in models:
    model_dir = os.path.join(cache_dir, model_name.replace("/", "-"))
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
        print(f"Deleted: {model_dir}")
    else:
        print(f"Not found: {model_dir}")


ImportError: cannot import name 'hf_cache_home' from 'transformers.utils' (c:\Users\prath\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\__init__.py)